In [5]:
# assumes that all are the same, i.e. same for both species and same high and low

import scipy.constants as const
import yaml

yamlfile = "input.yaml"

with open(yamlfile, 'r') as file:
    config = yaml.safe_load(file)

T = config['T']

epsilon_from_yaml = config['particle_types']['C']['epsilon_lo']
epsilon_kcal_per_mol = epsilon_from_yaml * const.Boltzmann * T * const.Avogadro / 4184

sigma = config['particle_types']['C']['sigma_lo']
LJcutoff = config['particle_types']['C']['cutoff_lo']
low = config['particle_types']['C']['low']
high = config['particle_types']['C']['high']


contentsuptoT = """\
variable cutoff index 15

units       real
dimension   3
boundary    p p p

atom_style  full

pair_style lj/cut/coul/long ${cutoff}
kspace_style pppm 1.0e-5
pair_modify     table 0

read_data       ./initialMD.data

group co2 type 1 2
group groupC type 1
group groupO type 2
group n2 type 3 4 
group groupX type 3
group groupN type 4


# --------------------- defining the potential (TraPPE) ---------------------

dielectric   1.0

pair_modify shift yes

pair_coeff 1 1 0.053654828567 2.8000
pair_coeff 2 2 0.156990053954 3.0500
pair_coeff 3 3 0.0 0.0
pair_coeff 4 4 0.07153943229  3.31

pair_coeff 1 2 0.091778398502 2.9250
pair_coeff 1 3 0.0 0.0
pair_coeff 1 4 0.06195511258 3.055
pair_coeff 2 3 0.0 0.0
pair_coeff 2 4 0.10597631497 3.18
pair_coeff 3 4 0.0 0.0

# --------------------- simulation/thermostat details ---------------------

neighbor 2.0 bin
neigh_modify delay 5 every 1 check yes

run_style verlet

timestep      0.5 # for equilibration, later set to 1.0

"""

line_containing_T = f"""\
variable      temperature equal {T}

"""


contents_between_T_and_LJ = """\
variable      timeequil equal  80000
variable      timerun equal   10000000
variable      tdamp equal  200.0


# --------------------- output style ---------------------

thermo           10000
thermo_style     custom step temp pe ke enthalpy lx vol press 
thermo_modify flush yes


# --------------------- initialise  ---------------------

velocity all create ${temperature} 432567 dist uniform loop geom

fix NVT all rigid/nvt molecule temp ${temperature} ${temperature} ${tdamp}


# --------------------- add external potential  ---------------------

"""

LJ_lines = f"""\
fix walllo all wall/lj93 zlo {low} {epsilon_kcal_per_mol} {sigma} {LJcutoff} units box pbc yes
fix wallhi all wall/lj93 zlo {high} {epsilon_kcal_per_mol} {sigma} {LJcutoff} units box pbc yes

"""

contents_after_LJ = """\
variable        z_iter  file ./zbins.txt
variable        forceO  file ./forceO.txt
variable        forceX  file ./forceX.txt
variable        forceC  file ./forceC.txt
variable        forceN  file ./forceN.txt
variable        imax equal 2000
variable        i loop ${imax}

label           start_of_loop_1
variable        binlo equal ${z_iter}
next            z_iter
region          reg_ez${i} block INF INF INF INF ${binlo} ${z_iter} units box
fix             addforceO${i} groupO addforce 0.0 0.0 ${forceO} region reg_ez${i}
fix             addforceX${i} groupX addforce 0.0 0.0 ${forceX} region reg_ez${i}
fix             addforceC${i} groupC addforce 0.0 0.0 ${forceC} region reg_ez${i}
fix             addforceN${i} groupN addforce 0.0 0.0 ${forceN} region reg_ez${i}
next            forceO
next            forceX
next            forceC
next            forceN
next            i
jump            SELF start_of_loop_1

velocity all zero linear

# --------------------- run equilibration  ---------------------

run          ${timeequil}

reset_timestep 0
timestep 1.0

# --------------------- collect density profiles and trajectory  ---------------------

compute ave_groupX groupX chunk/atom bin/1d z 0.0 0.02
fix avegroupX groupX ave/chunk 1 1000 1000 ave_groupX density/number norm sample file md_densityX.out ave running overwrite

compute ave_groupO groupO chunk/atom bin/1d z 0.0 0.02
fix avegroupO groupO ave/chunk 1 1000 1000 ave_groupO density/number norm sample file md_densityO.out ave running overwrite

compute ave_groupC groupC chunk/atom bin/1d z 0.0 0.02
fix avegroupC groupC ave/chunk 1 1000 1000 ave_groupC density/number norm sample file md_densityC.out ave running overwrite

compute ave_groupN groupN chunk/atom bin/1d z 0.0 0.02
fix avegroupN groupN ave/chunk 1 1000 1000 ave_groupN density/number norm sample file md_densityN.out ave running overwrite

dump 1 all custom 10000 md.lammpstrj id type xu yu zu

# ------------------------------- run -----------------------------

run          ${timerun}
"""

if epsilon_from_yaml != 0.0:
    all_contents = contentsuptoT + line_containing_T + contents_between_T_and_LJ + LJ_lines + contents_after_LJ
else:
    all_contents = contentsuptoT + line_containing_T + contents_between_T_and_LJ + contents_after_LJ

with open("in.lammps", "w") as f:
    f.write(all_contents)    